**Greedy Best First Search:** “I’ll always go to the neighbor that looks closest to the goal.”

**A-Star Search:** “I’ll go to the neighbor that best balances how far I’ve come and how near I still am.”

In [1]:
# ─── Necessary Imports ────────────────────────────────────────────────────────

import copy
import heapq
import time
from IPython.display import clear_output

In [2]:
# ─── Utility Functions ─────────────────────────────────────────────────────────

def clear_screen():
    """Clear Jupyter cell output."""
    clear_output(wait=True)

def display_board(board):
    """Display a 3×3 8-puzzle board in a nice grid."""
    print("+---+---+---+")
    for row in board:
        print("|", end="")
        for cell in row:
            tile = " " if cell == 0 else str(cell)
            print(f" {tile} |", end="")
        print("\n+---+---+---+")
    print()

def find_zero(board):
    """Find the (i,j) coordinates of the blank (0) tile."""
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return i, j
    raise ValueError("No zero found in board.")

def board_to_tuple(board):
    """Convert 2D board to a hashable tuple-of-tuples."""
    return tuple(tuple(row) for row in board)

In [3]:
# ─── Move Functions ────────────────────────────────────────────────────────────

def move_up(state):
    x, y = find_zero(state)
    if x == 0:
        return None
    new_state = copy.deepcopy(state)
    new_state[x][y], new_state[x-1][y] = new_state[x-1][y], new_state[x][y]
    return new_state

def move_down(state):
    x, y = find_zero(state)
    if x == 2:
        return None
    new_state = copy.deepcopy(state)
    new_state[x][y], new_state[x+1][y] = new_state[x+1][y], new_state[x][y]
    return new_state

def move_left(state):
    x, y = find_zero(state)
    if y == 0:
        return None
    new_state = copy.deepcopy(state)
    new_state[x][y], new_state[x][y-1] = new_state[x][y-1], new_state[x][y]
    return new_state

def move_right(state):
    x, y = find_zero(state)
    if y == 2:
        return None
    new_state = copy.deepcopy(state)
    new_state[x][y], new_state[x][y+1] = new_state[x][y+1], new_state[x][y]
    return new_state

def get_valid_moves(board):
    """Return a list of move functions valid from this board."""
    x, y = find_zero(board)
    moves = []
    if x > 0: moves.append(move_up)
    if x < 2: moves.append(move_down)
    if y > 0: moves.append(move_left)
    if y < 2: moves.append(move_right)
    return moves

In [4]:
# ─── Heuristic ─────────────────────────────────────────────────────────────────

def compute_goal_positions(goal):
    """Precompute each tile’s goal (i,j)."""
    pos = {}
    for i in range(3):
        for j in range(3):
            pos[goal[i][j]] = (i, j)
    return pos

def manhattan_heuristic(board, goal_positions):
    """Sum of Manhattan distances of all tiles from their goal."""
    dist = 0
    for i in range(3):
        for j in range(3):
            tile = board[i][j]
            if tile != 0:
                gi, gj = goal_positions[tile]
                dist += abs(i - gi) + abs(j - gj)
    return dist

In [5]:
# ─── A* Search ────────────────────────────────────────────────────────────────

def astar(start, goal):
    """Perform A* search from start to goal, animating each step."""
    goal_positions = compute_goal_positions(goal)
    start_h = manhattan_heuristic(start, goal_positions)
    
    # frontier: min-heap of (f, g, board, path_so_far)
    frontier = [(start_h, 0, start, [])]
    visited = { board_to_tuple(start): 0 }

    while frontier:
        f, g, current, path = heapq.heappop(frontier)

        clear_screen()
        print(f"g = {g}, h = {f - g}, f = {f}")
        display_board(current)
        time.sleep(2)

        if current == goal:
            print(f"🎉 Goal reached in {g} moves!")
            return path + [current]

        for move in get_valid_moves(current):
            neighbor = move(current)
            if not neighbor:
                continue
            ng = g + 1
            t = board_to_tuple(neighbor)

            # If this path to neighbor is better than any seen before
            if t not in visited or ng < visited[t]:
                visited[t] = ng
                nh = manhattan_heuristic(neighbor, goal_positions)
                heapq.heappush(frontier, (ng + nh, ng, neighbor, path + [current]))

    print("❌ No solution found.")
    return None

In [7]:
starting_node = [[0, 1, 3], [4, 2, 5], [7, 8, 6]]
goal_node = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

astar(starting_node, goal_node)

g = 4, h = 0, f = 4
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 |   |
+---+---+---+

🎉 Goal reached in 4 moves!


[[[0, 1, 3], [4, 2, 5], [7, 8, 6]],
 [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 0, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 0], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 6], [7, 8, 0]]]

***
***
***